In [1]:
import pandas as pd
import os
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict  # For word frequency
from gensim import utils
import numpy as np

In [2]:
list_en_fr = ['ES-GL', 'ES-EO', 'CA-FR', 'EN-EU', 'ES-OC', 'EO-CA', 'FR-ES', 'ES-FR', 'OC-ES', 'EN-ES', 'EN-GL', 'OC-CA', 'EO-FR', 'EN-CA', 'ES-CA', 'GL-EN', 'OC-FR', 'EO-ES', 'CA-ES', 'EN-EO', 'EU-EN', 'CA-EO', 'EO-EN', 'FR-EO', 'ES-EN', 'FR-CA', 'CA-OC', 'CA-EN', 'EU-ES', 'ES-EU', 'FR-OC', 'GL-ES']
def check_satisfy_en_fr(path):
    for x in list_en_fr:
        if x in path:
            return True
    return False

In [3]:
lists = os.listdir('../Small')
lists = [x for x in lists if x.endswith('.csv')]
lists = [x for x in lists if check_satisfy_en_fr(x)]
# mypd = [pd.read_csv('./Small/' + x) for x in lists]

In [4]:
dictionary = pd.read_csv('../EvaluationDic/En-Frv2.csv', error_bad_lines=False)

In [6]:
dic_source = dictionary['source'].tolist()

In [5]:
array = []
for x in lists[0:1]:
    df = pd.read_csv('../Small/' + x)
    for k in df['source'].tolist():
        array.append(k)

In [8]:
(list(set(array) - set(dic_source)))[0:15]

['Coloni',
 'Atienza',
 'Viñas',
 'réfrigérateur',
 'Marqués',
 'Cavassa',
 'Boquillon',
 'Gagnière',
 'Llinas',
 'Wundt',
 'Soriero',
 'holocène',
 'féminin',
 'con spirito',
 'barème']

In [6]:
list.sort(lists)
for x in lists:
    print(x)

TransSetEN-CA_small.csv
TransSetEN-ES_small.csv
TransSetEN-GL_small.csv
TransSetEO-CA_small.csv
TransSetEO-EN_small.csv
TransSetEO-ES_small.csv
TransSetEO-FR_small.csv
TransSetES-CA_small.csv
TransSetES-GL_small.csv
TransSetEU-EN_small.csv
TransSetEU-ES_small.csv
TransSetFR-CA_small.csv
TransSetFR-ES_small.csv
TransSetOC-CA_small.csv
TransSetOC-ES_small.csv
TransSetOC-FR_small.csv


In [12]:
def get_source_target(x):
    source = x.split('Set')[1].split('-')[0]
    target = x.split('Set')[1].split('-')[1].split('_')[0]
    return source, target

In [17]:
def df_to_sentense(file):
    df = pd.read_csv('../Small/' + file)
    source, target = get_source_target(file) 
    df['source'] = df['source'].apply(lambda x: "_".join(str(x).split()))
    df['target'] = df['target'].apply(lambda x: "_".join(str(x).split()))
    df['source_join'] = df['source']+'_'+ source + '_' + df['POS']
    df['target_join'] = df['target']+'_'+ target + '_' + df['POS']
    df['translate1'] = df['source_join'] + ' ' + df['target_join'] + ' ' + df['source_join'] +' ' + df['target_join']
    # df['translate2'] = df['target'] +'_'+ df['POS'] + ' ' + df['source'] + '_' + df['POS'] 
    # df['translate2'] = df['translate2'].astype(str)
#     array = [x.split() for x in df['translate1'].tolist()]
    array = [x for x in df['translate1'].tolist()]
    return array#" ".join(df['translate1']) #+ " ".join(df['translate2'])

In [18]:
sent = [df_to_sentense(x) for x in lists]

In [24]:
# write sent to file:
with open('../data_train/data_train.txt', 'w') as f:
    for a in sent:
        for item in a:
            f.write("%s\n" % item)

In [61]:
sent = [df_to_sentense(x) for x in lists]
#sent = [row.split() for row in sent]

#create corpus by joint all line in sent
corpus = []
for se in sent:
    for line in se:
        corpus.append(line)

In [75]:
model = Word2Vec(sentences=corpus, size=300, iter=300, window=1, min_count=4)

In [76]:
model.wv.vectors.shape

(203004, 300)

In [261]:
model.save('./model/first_model.model')

In [81]:
model.wv.similar_by_word('free_adjective')

[('doako_adjective', 0.992991030216217),
 ('gratuíto_adjective', 0.9928751587867737),
 ('gratuito_adjective', 0.9925113916397095),
 ('aske_adjective', 0.9909542202949524),
 ('senpaga_adjective', 0.9906609058380127),
 ('lliure_adjective', 0.9890164732933044),
 ('gratuit_adjective', 0.988632082939148),
 ('liure_adjective', 0.9878025054931641),
 ('lineare_nedependa_adjective', 0.9868024587631226),
 ('gratuït_adjective', 0.9865648746490479)]

In [98]:
model.wv[['crumb_noun', 'free_adjective']][0:2,0:2]

array([[-0.03385356,  0.13950163],
       [ 0.23350666, -0.04644945]], dtype=float32)

In [99]:
model.wv['crumb_noun'][0:2]

array([-0.03385356,  0.13950163], dtype=float32)

In [101]:
path_dict_fr = ['TransSetFR-CA_small.csv', 'TransSetFR-ES_small.csv']
df_fr1 = pd.read_csv('./Small/' + path_dict_fr[0])
df_fr2 = pd.read_csv('./Small/' + path_dict_fr[1])

In [148]:
a = (df_fr1['source'].astype(str) + '_' + df_fr1['POS'].astype(str)).to_list()
b = (df_fr2['source'].astype(str) + '_' + df_fr2['POS'].astype(str)).to_list()

In [164]:
vocabulary = list(model.wv.vocab.keys())

In [165]:
france_dict = list(set(a) & set(b))
france_dict = list(set(france_dict) & set(vocabulary))
france_vectors = model[france_dict]

<ipython-input-165-daf035289aa7>:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  france_vectors = model[france_dict]


In [157]:
def to_word(list_df):
    words = []
    for path in list_df:
        df = pd.read_csv('./Small/' + path)
        words += (df['source'].astype(str) + '_' + df['POS'].astype(str)).to_list()
    return set(words)

In [162]:
path_dict_en = ['TransSetEN-CA_small.csv', 'TransSetEN-ES_small.csv', 'TransSetEN-GL_small.csv']
words_en = to_word(path_dict_en)

In [166]:
words_en = list(set(words_en) & set(vocabulary))
english_vectors = model[words_en]

<ipython-input-166-35fcd3405bfb>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  english_vectors = model[words_en]


In [178]:
english_vectors.shape

(41145, 300)

In [175]:
def get_pos(word):
    return word.split('_')[-1]

In [186]:
def find_translate(word_vec, matrix_vec, dic_trans, pos):
    assert len(matrix_vec) == len(dic_trans)
    cosine_similary = model.wv.cosine_similarities(word_vec, matrix_vec)
    sort_cosine = np.argsort(-cosine_similary)
    for index in sort_cosine:
        trans_word = dic_trans[index]
        if(get_pos(trans_word)==pos):
            return trans_word

In [259]:
dict_en_fr = []
for word in words_en:
    pos = get_pos(word)
    word_vec = model[word]
    trans_word = find_translate(word_vec, france_vectors, france_dict, pos)
    if trans_word != None:
        trans_word = to_word(trans_word)
    dict_en_fr.append(to_word(word)+'\t'+str(trans_word)+'\t'+str(pos)+'\t1')

<ipython-input-259-497a10250b79>:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word_vec = model[word]


In [231]:
def to_word(word):
    return ' '.join(word.split('_')[0:-1])

In [234]:
dict_en_fr.insert(0, '')

41145

In [260]:
with open('./BuildDictionary/TIADbaseline_trans_en-fr.tsv', 'w') as f:
    for item in dict_en_fr:
        f.write("%s\n" % item)